### 기본코드

In [1]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor

training_data = datasets.FashionMNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor()
)

test_data = datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor()
)

train_dataloader = DataLoader(training_data, batch_size=64)
test_dataloader = DataLoader(test_data, batch_size=64)

class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10),
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

model = NeuralNetwork()

  0%|          | 0/26421880 [00:00<?, ?it/s]

Extracting data\FashionMNIST\raw\train-images-idx3-ubyte.gz to data\FashionMNIST\raw



  0%|          | 0/29515 [00:00<?, ?it/s]

Extracting data\FashionMNIST\raw\train-labels-idx1-ubyte.gz to data\FashionMNIST\raw



  0%|          | 0/4422102 [00:00<?, ?it/s]

Extracting data\FashionMNIST\raw\t10k-images-idx3-ubyte.gz to data\FashionMNIST\raw



  0%|          | 0/5148 [00:00<?, ?it/s]

Extracting data\FashionMNIST\raw\t10k-labels-idx1-ubyte.gz to data\FashionMNIST\raw



### 하이퍼파라미터

In [2]:
learning_rate = 1e-3
batch_size = 64
epochs = 5

### 손실함수

In [3]:
loss_fn = nn.CrossEntropyLoss()

### 옵티마이저

In [4]:
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

### 전체 구현

In [5]:
def train_loop(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    for batch, (X, y) in enumerate(dataloader):  # dataloader에서 배치씩 반복함
        # 예측(prediction)과 손실(loss) 계산
        pred = model(X) # 예측값
        loss = loss_fn(pred, y) # loss값

        # 역전파
        optimizer.zero_grad() # zero_grad():grad값들이 더해지는 것을 막기 위해 루프돌 때 마다 역전파 하기전에 grad 값들을 0으로 초기화 시킨 후 학습 진행 
        loss.backward() # 역전파
        optimizer.step() # 역전파에서 수집된 grad로 매개변수 조정

        if batch % 100 == 0:
            loss, current = loss.item(), batch * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")


def test_loop(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    test_loss, correct = 0, 0

    with torch.no_grad():
        for X, y in dataloader:
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()

    test_loss /= num_batches # test_loss/num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [6]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

epochs = 10
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train_loop(train_dataloader, model, loss_fn, optimizer)
    test_loop(test_dataloader, model, loss_fn)
print("Done!")

Epoch 1
-------------------------------
loss: 2.306095  [    0/60000]
loss: 2.299719  [ 6400/60000]
loss: 2.279140  [12800/60000]
loss: 2.268825  [19200/60000]
loss: 2.247615  [25600/60000]
loss: 2.217066  [32000/60000]
loss: 2.231867  [38400/60000]
loss: 2.193394  [44800/60000]
loss: 2.190137  [51200/60000]
loss: 2.157220  [57600/60000]
Test Error: 
 Accuracy: 29.4%, Avg loss: 2.152627 

Epoch 2
-------------------------------
loss: 2.166462  [    0/60000]
loss: 2.166233  [ 6400/60000]
loss: 2.105104  [12800/60000]
loss: 2.116839  [19200/60000]
loss: 2.073460  [25600/60000]
loss: 2.009911  [32000/60000]
loss: 2.040689  [38400/60000]
loss: 1.962233  [44800/60000]
loss: 1.966599  [51200/60000]
loss: 1.895323  [57600/60000]
Test Error: 
 Accuracy: 52.3%, Avg loss: 1.888531 

Epoch 3
-------------------------------
loss: 1.931108  [    0/60000]
loss: 1.903749  [ 6400/60000]
loss: 1.781796  [12800/60000]
loss: 1.816402  [19200/60000]
loss: 1.709342  [25600/60000]
loss: 1.665423  [32000/600